In [1]:
import os
import shutil
import numpy as np
import pandas as pd
from pandas import ExcelWriter
from allensdk.brain_observatory.ecephys.ecephys_session import EcephysSession
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from keras.layers import Dense, Dropout, LSTM, BatchNormalization, Flatten, TimeDistributed
from keras.models import Sequential
from keras.utils import to_categorical
import tensorflow as tf
from keras.optimizers import Adam
import decoding_functions

Using TensorFlow backend.


In [2]:
data_dir = 'ecephys_cache_dir'
manifest_path = os.path.join(data_dir, 'manifest.json')
cache = EcephysProjectCache.from_warehouse(manifest = manifest_path)
session_table = cache.get_session_table()

In [8]:
all_sessions = [719161530]
session_id = all_sessions[0]
session = cache.get_session_data(session_id)

In [9]:
## Copied over from decodeAlien.py
scene_presentations = decoding_functions.fetch_scenes(session)
areas = ['VISp']
units = decoding_functions.fetch_units(session,areas)

# Bin duration in ms as input, leave the rest alone (will calculate value in seconds and pass to appropriate functions
bin_duration_in_msec = 25

X_hold, X = decoding_functions.fetch_x(scene_presentations,units,session,bin_duration_in_seconds=bin_duration_in_msec/1000)
Y = scene_presentations.loc[X_hold.stimulus_presentation_id.values, 'frame']

X_train, X_test, X_valid, Y_train, Y_test, Y_valid = decoding_functions.prepare_data(X, Y)

[0.    0.025 0.05  0.075 0.1   0.125 0.15  0.175 0.2   0.225 0.25 ]


In [56]:
from sklearn import svm
import pandas as pd

## Setup SVM
## reshape X_train to fit into 2D-requiring SVM
tx = int(0.7*len(X))
Y_adj = Y[:tx]
Y_valid_adj = Y[(int(0.85*len(Y))):]
Y_test_adj = Y[int(0.7*len(Y)):(int(0.85*len(Y)))]
Y_adju = Y_adj.astype('int')
Y_test_adj2 = Y_test_adj.astype('int')
X_train_re = np.reshape(X_train, (3656, 10*308))
X_test_re = np.reshape(X_test, (784, 10*308))
X_val_re = np.reshape(X_valid, (784, 10*308))

clf = svm.SVC(kernel='linear')
clf.fit(X_train_re, Y_adju)
y_pred = clf.predict(X_test_re)
#clf.score(X_val_re, Y_valid_adj)
clf.score(X_test_re,Y_test_adj2)

0.7691326530612245

In [24]:
X_train_re = np.reshape(X_train, (3656, 10*308))
np.shape(X_train_re)
Y[:tx]

stimulus_presentation_id
51353      5
51354    114
51355     13
51356     70
51357     26
        ... 
66654     22
66655      7
66656     80
66657     20
66658     53
Name: frame, Length: 3656, dtype: object

In [34]:
## fetch units
areas = ['VISp', 'DG', 'LP']
units2new = session.units[session.units['ecephys_structure_acronym'] ==areas[0]]
for sp_area in range(len(areas[1:])):

    units2 = [session.units[session.units['ecephys_structure_acronym'] == areas[sp_area+1]]]
              
    # units2new = units2new.append(units2)
    units2new = units2new.append(units2)
    continue
units2new.index.values[:]

array([950930145, 950930105, 950930276, 950933698, 950930237, 950930215,
       950930358, 950930340, 950930437, 950930423, 950930407, 950930392,
       950930375, 950930522, 950930454, 950933732, 950930866, 950930795,
       950930888, 950931118, 950931004, 950930985, 950934181, 950930964,
       950931043, 950931236, 950931181, 950931164, 950931272, 950931254,
       950931423, 950931363, 950933840, 950931458, 950931315, 950931565,
       950931533, 950931517, 950931656, 950931805, 950931617, 950931581,
       950931770, 950931751, 950931727, 950931853, 950931959, 950931899,
       950931878, 950933890, 950932102, 950932087, 950933907, 950932032,
       950933924, 950932445, 950932578, 950932563, 950932696, 950933960,
       950911006, 950911040, 950911088, 950926709, 950926867, 950926928,
       950926886, 950927046, 950927024, 950927002, 950926805, 950926787,
       950927066, 950927341, 950927229, 950937068, 950937114, 950937333,
       950952985, 950949089, 950949200, 950949118, 

In [35]:
units = pd.concat([session.units[session.units['ecephys_structure_acronym'] == 'VISp'],
                       session.units[session.units['ecephys_structure_acronym'] == 'DG'],
                       session.units[session.units['ecephys_structure_acronym'] == 'LP']])
units.index.values

array([950930145, 950930105, 950930276, 950933698, 950930237, 950930215,
       950930358, 950930340, 950930437, 950930423, 950930407, 950930392,
       950930375, 950930522, 950930454, 950933732, 950930866, 950930795,
       950930888, 950931118, 950931004, 950930985, 950934181, 950930964,
       950931043, 950931236, 950931181, 950931164, 950931272, 950931254,
       950931423, 950931363, 950933840, 950931458, 950931315, 950931565,
       950931533, 950931517, 950931656, 950931805, 950931617, 950931581,
       950931770, 950931751, 950931727, 950931853, 950931959, 950931899,
       950931878, 950933890, 950932102, 950932087, 950933907, 950932032,
       950933924, 950932445, 950932578, 950932563, 950932696, 950933960,
       950911006, 950911040, 950911088, 950926709, 950926867, 950926928,
       950926886, 950927046, 950927024, 950927002, 950926805, 950926787,
       950927066, 950927341, 950927229, 950937068, 950937114, 950937333,
       950952985, 950949089, 950949200, 950949118, 

In [27]:
areas = ['VISp', 'DG', 'LP']
units2new = session.units[session.units['ecephys_structure_acronym'] ==areas[0]]
units2=[]
for sp_area in range(len(areas)):

    units2[sp_area] = [session.units[session.units['ecephys_structure_acronym'] == areas[sp_area]]]
              
    # units2new = units2new.append(units2)
    #units2new = pd.concat(units2new, units2)
    continue
units2

IndexError: list assignment index out of range

In [ ]:
session2=cache.get_session_data(791319847)
session2.structurewise_unit_counts

In [ ]:
session2=cache.get_session_data(791319847)
session2.structurewise_unit_counts

In [77]:
train_test_svm = int(0.7*len(X))
yyz = np.reshape(X[:train_test_svm], (np.shape(X[:train_test_svm])[0], ((np.shape(X[:train_test_svm])[1])*(np.shape(X[:train_test_svm])[2]))))
#np.shape(X)
np.shape(yyz)

(3656, 3080)

In [78]:

pd.DataFrame(clf.score(X_test_re,Y_test_adj2))

ValueError: DataFrame constructor not properly called!

In [13]:
Y_cat = to_categorical(Y, num_classes=119)
np.shape(Y_cat)

(5950, 119)

In [15]:
all_sessions = session_table.loc[session_table.session_type == 'brain_observatory_1.1'].index
all_sessions[-4:]


Int64Index([791319847, 797828357, 798911424, 799864342], dtype='int64', name='id')